In [1]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/ultrachat_200k")

save_path = r"C:\Users\ramos\Desktop\GitHub\BDLab-Agent\backend\data\LLMs\datasets\ultrachat"
print(f"Saving to {save_path}")
dataset.save_to_disk(save_path)

c:\Users\ramos\Desktop\GitHub\BDLab-Agent\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Saving to C:\Users\ramos\Desktop\GitHub\BDLab-Agent\backend\data\LLMs\datasets\ultrachat


Saving the dataset (1/1 shards): 100%|██████████| 28304/28304 [00:00<00:00, 295505.72 examples/s]


In [ ]:
import json

# review the type, length, and structure of the model.
print(type(dataset))
print(len(dataset))
print(dataset)

<class 'datasets.dataset_dict.DatasetDict'>
4
DatasetDict({
    train_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 207865
    })
    test_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 23110
    })
    train_gen: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 256032
    })
    test_gen: Dataset({
        features: ['prompt', 'prompt_id', 'messages'],
        num_rows: 28304
    })
})


In [5]:
sft_train = dataset["train_sft"]
print(json.dumps(sft_train[0], indent=2))

{
  "prompt": "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
  "prompt_id": "f0e37e9f7800261167ce91143f98f511f768847236f133f2d0aed60b444ebe57",
  "messages": [
    {
      "content": "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by

In [6]:
import torch
from pathlib import Path
from transformers import AutoModelForCausalLM, AutoTokenizer

base = Path(r"C:\Users\ramos\Desktop\GitHub\BDLab-Agent\backend\data\LLMs\Phi3-mini")

Phi_3 = AutoModelForCausalLM.from_pretrained(
    str(base),
    #dtype=torch.bfloat16,
    device_map="auto",
    local_files_only=True)

Phi_3_tokenizer = AutoTokenizer.from_pretrained(str(base),local_files_only=True)
print(Phi_3_tokenizer.chat_template)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.55s/it]
Some parameters are on the meta device because they were offloaded to the disk and cpu.


{% for message in messages %}{% if message['role'] == 'system' %}{{'<|system|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'user' %}{{'<|user|>
' + message['content'] + '<|end|>
'}}{% elif message['role'] == 'assistant' %}{{'<|assistant|>
' + message['content'] + '<|end|>
'}}{% endif %}{% endfor %}{% if add_generation_prompt %}{{ '<|assistant|>
' }}{% else %}{{ eos_token }}{% endif %}


In [8]:
# the function abstracted inside SFTTrainer. (located inside sample_finetune.py "from trl import SFTTrainer")

formatted_text = Phi_3_tokenizer.apply_chat_template(
    sft_train[0]["messages"], 
    tokenize=False, 
    add_generation_prompt=False
)

print(formatted_text)

<|user|>
These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?
On your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!
Your Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.
Does this feature apply to all sections of the theme or just specific ones as listed in the text material?<|end|>
<|assistant|>
This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.<|end|>
<|user|>
Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?<|end|>
<|assistant|>
Sure, here are the steps to enable the secondary image hover feature on yo

### Key points:

- apply_chat_template: Converts a list of message dictionaries into a single string.
- Delimiters: Inserts specific markers (<|user|>, <|assistant|>, <|end|>) to define conversational boundaries.
- Configuration: The parameters ensure the output represents a full dialogue history for training rather than a prompt for a new response.

### Purpose in Training

This formatting allows the SFTTrainer to teach the model to distinguish between user input and its own responses. By identifying these markers, the model learns the correct sequence for multi-turn interactions.